## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-11-22-47-07 +0000,nyt,New York’s Democratic Governor Called in the T...,https://www.nytimes.com/2025/08/11/us/politics...
1,2025-08-11-22-46-53 +0000,cbc,Gunman fired more than 180 shots at CDC headqu...,https://www.cbc.ca/news/world/cdc-headquarters...
2,2025-08-11-22-42-39 +0000,nyt,"Who is Lip-Bu Tan, the Intel CEO in Trump’s Cr...",https://www.nytimes.com/2025/08/11/technology/...
3,2025-08-11-22-41-56 +0000,nypost,Zohran Mamdani blasts Cuomo plan to block priv...,https://nypost.com/2025/08/11/us-news/zohran-m...
4,2025-08-11-22-37-53 +0000,nypost,NYC designer Martha Nolan-O’Slatarra finalized...,https://nypost.com/2025/08/11/us-news/nyc-desi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2316/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,trump,60
84,china,18
0,new,14
38,he,13
153,al,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
85,2025-08-11-19-43-00 +0000,wsj,The Trump administration’s decision to charge ...,https://www.wsj.com/politics/national-security...,137
42,2025-08-11-21-29-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...,136
155,2025-08-11-14-54-28 +0000,nypost,"Trump takes control of DC police, deploying Na...",https://nypost.com/2025/08/11/us-news/trump-fe...,122
25,2025-08-11-22-10-03 +0000,startribune,Trump says he's placing Washington police unde...,https://www.startribune.com/trump-is-promising...,116
253,2025-08-11-01-00-00 +0000,wsj,News the jobs market was much weaker than prev...,https://www.wsj.com/economy/jobs-inflation-tru...,115


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
85,137,2025-08-11-19-43-00 +0000,wsj,The Trump administration’s decision to charge ...,https://www.wsj.com/politics/national-security...
42,68,2025-08-11-21-29-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...
41,49,2025-08-11-21-31-11 +0000,nypost,"Crypto bro accused of threatening ex-fiancée, ...",https://nypost.com/2025/08/11/us-news/crypto-b...
16,46,2025-08-11-22-21-40 +0000,nyt,Who Were the Al Jazeera Journalists Killed in ...,https://www.nytimes.com/2025/08/11/world/middl...
139,39,2025-08-11-16-09-47 +0000,nypost,Trump says he’ll tell Putin to end Ukraine inv...,https://nypost.com/2025/08/11/world-news/trump...
3,38,2025-08-11-22-41-56 +0000,nypost,Zohran Mamdani blasts Cuomo plan to block priv...,https://nypost.com/2025/08/11/us-news/zohran-m...
193,38,2025-08-11-11-55-53 +0000,nypost,Colombian presidential hopeful Miguel Uribe di...,https://nypost.com/2025/08/11/world-news/colom...
151,37,2025-08-11-15-03-55 +0000,nyt,"Netflix Signs New, Looser Deal With Prince Har...",https://www.nytimes.com/2025/08/11/business/me...
152,37,2025-08-11-15-02-46 +0000,nypost,Wild video shows Chinese ships crashing while ...,https://nypost.com/2025/08/11/world-news/chine...
57,34,2025-08-11-21-00-30 +0000,bbc,US Steel plant explosion in Pennsylvania leave...,https://www.bbc.com/news/articles/ckgjj49neldo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
